In [1]:
%load_ext autoreload
%autoreload 2

## Medchem

Medchem is a package for applying general filtering rules on a set of molecules to ensure they have drug-like properties.

In this tutorial, we will apply various filtering on an example dataset to get highlight the package API

### Setup

In [11]:
import datamol as dm
import random
import numpy as np
from loguru import logger

data = dm.data.freesolv().sample(100)
smiles_list = data.smiles.values

In [12]:
from medchem.filter import lead
from medchem.demerits import score
from medchem.novartis import NovartisFilters
from medchem.alerts import AlertFilters

### Using the filter module

The filter module provides a variety of two types of filters:
-  `generic`: custom filtering based on some given molecule properties such as number of atoms, presence of specific atom type, etc
-  `lead`: filtering based on structural motifs that are known to either be toxic, reactive, unstable or frequent false positive

In [13]:
# common filters including pains, brenk, nih, zinc
lead.common_filter(smiles_list, brenk=True, nih=True)

array([ True, False, False, False, False, False, False, False, False,
       False, False,  True,  True, False, False,  True, False,  True,
       False, False,  True, False,  True, False, False,  True, False,
       False, False,  True, False, False,  True,  True,  True,  True,
        True,  True, False,  True, False,  True,  True, False, False,
        True,  True, False,  True,  True, False, False, False,  True,
        True, False, False,  True, False, False,  True, False,  True,
        True,  True, False, False, False,  True,  True,  True,  True,
       False, False,  True, False, False, False,  True, False, False,
        True, False, False,  True,  True,  True, False, False, False,
       False, False,  True,  True, False, False, False,  True, False,
       False])

In [20]:
# filtering based on some commons alerts + additional lead like rules
lead.alert_filter(smiles_list, alerts=["Glaxo", "BMS"], rule_dict=dict(MW=[0, 100]))

array([False, False, False, False, False, False, False, False, False,
       False, False, False,  True,  True, False,  True, False, False,
       False, False, False,  True, False, False, False, False, False,
        True, False,  True, False, False, False,  True,  True, False,
        True, False, False, False, False, False, False, False, False,
       False,  True,  True,  True, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False,  True, False,
       False, False,  True, False, False,  True,  True, False, False,
       False, False, False, False, False,  True, False, False,  True,
       False,  True,  True, False, False, False, False,  True, False,
       False])

In [22]:
# filtering based on NIBR screening deck process described in
# "Evolution of Novartis' small molecule screening deck design" by Schuffenhauer, A. et al. J. Med. Chem. (2020), 
# https://dx.doi.org/10.1021/acs.jmedchem.0c01332. 
lead.screening_filter(smiles_list, return_idx=True)

array([ 0,  1,  2,  3,  4,  5,  6,  8,  9, 11, 12, 13, 15, 16, 17, 18, 19,
       20, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
       38, 39, 41, 42, 43, 45, 46, 48, 49, 50, 51, 53, 54, 55, 57, 58, 60,
       61, 62, 63, 64, 65, 67, 68, 69, 70, 71, 72, 73, 74, 75, 77, 78, 79,
       80, 81, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [37]:
# Filter based on the demerit scoring of Eli Lilly
test_config = {
    "output": "test",
    "min_atoms": 10,
    "soft_max_atoms": 30,
    "hard_max_atoms": 50,
    "smarts": [],
    "nodemerit": False,
    "dthresh": 160,
    "odm": [],
    "okiso": False,
    "noapdm": False,
}
lead.demerit_filter(smiles_list, max_demerits=160, return_idx=True, **test_config)



array([20, 35, 60, 62, 64, 69, 73, 79, 81, 85, 88, 96, 98])

### Advanced options

The advanced options allow a better control over the filtering process. They also provide more information on the issues with the molecules.

#### AlertFilter

These are the underlying filters called by `lead.alert_filter`. In the output, the compound status is indicated as either `"Exclude"` or `"Ok"`.

In [41]:
filter_obj = AlertFilters(alerts_set=["inpharmatica", "SureChEMBL"])
out = filter_obj(smiles_list)
out

,_smiles,status,reasons,MW,LogP,HBD,HBA,TPSA
0,CCOC(=O)CC,Ok,None,102.133,0.9595,0,2,26.30
1,CCCCCCO,Ok,None,102.177,1.5590,1,1,20.23
2,CC(C)CCOC=O,Exclude,formate_formide; Filter38_aldehyde,116.160,1.2055,0,2,26.30
3,COc1ccc(N)cc1,Exclude,analine,123.155,1.2774,1,2,35.25
4,Clc1cccc(Cl)c1Cl,Ok,None,181.449,3.6468,0,0,0.00
...,...,...,...,...,...,...,...,...
95,CCCCCc1ccccc1,Ok,None,148.249,3.4193,0,0,0.00
96,FC(F)[C@@H](F)Oc1ccccc1,Ok,None,176.137,2.6261,0,1,9.23
97,CN1CCCCC1,Ok,None,99.177,1.1021,0,1,3.24
98,COC(=O)c1ccc([N+](=O)[O-])cc1,Ok,None,181.147,1.3814,0,4,69.44


#### NovartisFilter

These are the underlying filters called by `lead.screening_filter`. 

Here is an explanation of the output:
- **status**: one of `["Exclude", "Flag", "Annotations", "Ok"]` (ordered by quality). Generally, you can keep anything without the "Exclude" label, as long as you also apply a maximum severity score for compounds that collects too many flags.
- **covalent**: number of potentially covalent motifs contained in the compound
- **severity**: how severe are the issues with the molecules:
      - `0`: compound has no flags, might have annotations;
      - `1-9`:  number of flags the compound raises;
      - `>= 10`:  default exclusion criterion used in the paper
- **special_mol**: whether the compound/parts of the compound belongs to a special class of molecules (e.g peptides, glycosides, fatty acid). In that case, you should review the rejection reasons.

In [43]:
filter_obj = NovartisFilters()
out = filter_obj(smiles_list)
out

,_smiles,status,reasons,severity,covalent,special_mol
0,CCOC(=O)CC,Ok,None,0,NaN,NaN
1,CCCCCCO,Ok,None,0,NaN,NaN
2,CC(C)CCOC=O,Ok,None,0,NaN,NaN
3,COc1ccc(N)cc1,Flag,aniline_no_alkyl_min(1),1,0.0,0.0
4,Clc1cccc(Cl)c1Cl,Flag,halogen_aromatic_count_3_min(3); polyhalogenat...,1,0.0,0.0
...,...,...,...,...,...,...
95,CCCCCc1ccccc1,Ok,None,0,NaN,NaN
96,FC(F)[C@@H](F)Oc1ccccc1,Annotations,alpha_halo_heteroatom_min(1),0,0.0,0.0
97,CN1CCCCC1,Ok,None,0,NaN,NaN
98,COC(=O)c1ccc([N+](=O)[O-])cc1,Annotations,nitro_count_1_min(1),0,0.0,0.0


#### Demerits scoring

Demerit scoring uses the Eli Lilly filter rules. Those are complex rules, that can be customized in any way you wish. 

The following "information" will be computed and added as columns to a DataFrame for each run:

- **status**: this was added for compatibility and has values `"Exclude"`, `"Flag"` or `"Ok"`.
- **rejected** : whether the molecule pass the filter or was rejected
- **reasons**: the reasons why the molecule was rejected if available
- **demerit_score** a demerit score for molecules. The lower the better. A cutoff is used to reject molecule with too many demerits, which you can refilter again after.
- **step**: step of the pipeline where molecule was filtered out, if available



In [47]:
out = score(smiles_list, **test_config)
out

,_smiles,ID,reasons,step,rejected,demerit_score,status
0,CCC(=O)OCC,0,not_enough_atoms,1,True,NaN,Exclude
1,CCCCCCO,1,not_enough_atoms,1,True,NaN,Exclude
2,CC(C)CCOC=O,2,not_enough_atoms,1,True,NaN,Exclude
3,COC1=CC=C(N)C=C1,3,not_enough_atoms,1,True,NaN,Exclude
4,C1=CC(=C(Cl)C(=C1)Cl)Cl,4,not_enough_atoms,1,True,NaN,Exclude
...,...,...,...,...,...,...,...
95,CCCCCC1=CC=CC=C1,95,no_interesting_atoms,1,True,NaN,Exclude
96,C1=CC=C(C=C1)O[C@H](F)C(F)F,96,NaN,4,False,0.0,Ok
97,CN1CCCCC1,97,not_enough_atoms,1,True,NaN,Exclude
98,COC(=O)C1=CC=C(C=C1)N(=O)=O,98,"nitro:D60,ester:D35",4,False,95.0,Flag
